# Machine Learning Experiment \#1

In [1]:
import pandas as pd
import numpy as np
import scipy as sp

import plotly.offline as py
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)

import ipywidgets

import datetime
import math
import bisect
import time

In [2]:
class ModelMarket:
    def __init__(self):
        self.curr_index = np.floor(np.random.rand()*10)
        self.position = False
        self.bought_price = None
        
    def get_price(self, index):
        period = 10
        ii = index % period
        #return ii
        if ii < period/2:
            return ii
        else:
            return period-ii
        
    def buy(self):
        if not self.position:
            self.position = True
            self.bought_price = self.get_price(self.curr_index)
        else:
            error('FUCK out of shares')
            
    def sell(self):
        if self.position:
            self.position = False
            reward = self.get_price(self.curr_index) - self.bought_price
            self.bought_price = None
            return reward
        else:
            error('FUCK out of shares')
            
    def tick(self):
        self.curr_index += 1
    

In [3]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras import optimizers
from keras import regularizers
import random
import numpy as np
import pandas as pd
from operator import add


class DQNAgent(object):

    def __init__(self):
        self.inputs = 2
        self.model = self.network()
        self.memory = []
        self.short_memory = []
        self.gamma = 0.9   
        
    def get_state(self, market):
        state = np.diff([market.get_price(market.curr_index-self.inputs+ii+1) for ii in range(self.inputs)])
        state = [max(x,0) for x in state]
        state = [1 if market.position else 0] + state
        
        return state

    def network(self, weights=None):
        model = Sequential()
        
        num_layers = 1
        neurons_per_layer = 4
        dropout = 0.0
        reg = 0.00
        act = 'tanh'
        output_act = 'linear'
        learning = 0.001
        opt = optimizers.Adam(learning)
        
        if num_layers == 0:
            if dropout > 0:
                    model.add(Dropout(dropout))
            model.add(Dense(units=2, activation=output_act, input_dim=self.inputs,
                        kernel_regularizer=regularizers.l1(reg),
                        activity_regularizer=regularizers.l1(reg)))
        else:
            model.add(Dense(units=neurons_per_layer, activation=act, input_dim=self.inputs,
                            kernel_regularizer=regularizers.l1(reg),
                            activity_regularizer=regularizers.l1(reg)))
            
            for ii in range(num_layers - 1):
                if dropout > 0:
                    model.add(Dropout(dropout))
                model.add(Dense(units=neurons_per_layer, activation=act,
                                kernel_regularizer=regularizers.l1(reg),
                                activity_regularizer=regularizers.l1(reg)))
                
            if dropout > 0:
                    model.add(Dropout(dropout))
            model.add(Dense(units=2, activation=output_act,
                            kernel_regularizer=regularizers.l1(reg),
                            activity_regularizer=regularizers.l1(reg))) 
    
        model.compile(loss='mse', optimizer=opt)

        return model

    def remember(self, state, action, reward, next_state):
        self.memory.append((state, action, reward, next_state))
        self.short_memory.append((state, action, reward, next_state))
        
        
        
    def train(self):
        if len(self.short_memory) > 1000:
            minibatch_short = random.sample(self.short_memory, 1000)
        else:
            minibatch_short = self.short_memory
        if len(self.memory) > 1000:
            minibatch_long = random.sample(self.memory, 1000)
        else:
            minibatch_long = self.memory
            
        minibatch = minibatch_short + minibatch_long
            
        state_array = len(minibatch) * [None]
        target_f_array = len(minibatch) * [None]   
        index = 0
        for state, action, reward, next_state in minibatch:
            target = reward + self.gamma * np.amax(self.model.predict(np.array([next_state]))[0])
            target_f = self.model.predict(np.array([state]))
            target_f[0][np.argmax(action)] = target
            
            state_array[index] = np.asarray(state)
            target_f_array[index] = target_f[0]
            
            #print('State ', state, ', Action ', action)
            #print('Reward ', reward, ', Target ', target, ', Targetf ', target_f)
            index += 1
        state_array = np.asarray(state_array)
        target_f_array = np.asarray(target_f_array)
        self.model.fit(state_array, target_f_array, epochs=1, verbose=0, batch_size = 22)
        self.short_memory = []
        
    def train_single(self, state, action, reward, next_state):
        target = reward + self.gamma * np.amax(self.model.predict(np.array([next_state]))[0])
        target_f = self.model.predict(np.array([state]))
        target_f[0][np.argmax(action)] = target
        self.model.fit(np.array([state]), target_f, epochs=1, verbose=0)
        
    def predict(self,state):
        return self.model.predict(np.asarray(state).reshape(1, self.inputs))

Using TensorFlow backend.


In [4]:
class Chart:
    def __init__(self):
        data1 = [dict(
                type = 'heatmap',
                x = [],
                y = [],
                z = [],
                zmin = -0.1,
                zmax = 0.1)]
        layout1 = dict(
                title = 'Var1',
                xaxis = dict(
                    title = dict(
                        text = 'x'
                    )
                ),
                yaxis = dict(
                    title = dict(
                        text = 'y'
                    )
                ))
        
#         data2 = [dict(
#                 type = 'heatmap',
#                 x = [],
#                 y = [],
#                 z = [])]
#         layout2 = dict(
#                 title = 'Var2',
#                 xaxis = dict(
#                     title = dict(
#                         text = 'x'
#                     )
#                 ),
#                 yaxis = dict(
#                     title = dict(
#                         text = 'y'
#                     )
#                 ))
        
        self.chart1 = go.FigureWidget( data=data1, layout=layout1)
#         self.chart2 = go.FigureWidget( data=data2, layout=layout2)
#         self.display = ipywidgets.HBox([self.chart1, self.chart2])
        
        self.display = self.chart1
        
chart = Chart()
chart.display

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': '321b22b4-572d-4e87-8232-15508587eb9c',
 …

In [ ]:
from keras.utils import to_categorical
from random import randint

def run(chart):
    agent = DQNAgent()
#     agent.model.layers[0].set_weights(
#         [np.array([[-0.4938594, -0.9615469],
#        [ 1.0600642,  1.1646739]]), np.array([0,0])]
#     )
#     agent.model.layers[1].set_weights(
#         [np.array([[-1.1078124 , -0.43855244],
#        [-0.32486475, -0.03094018]]), np.array([0,0])]
#     )
    
    num_tests = 500
    ticks_per_test = 100
    
    total_reward_old = -50
    
    for ii in range(num_tests):
        market = ModelMarket()
        epsilon = 0#(10)*(total_reward_old < 30)#200 - 10 * ii #randomness
        
        total_reward = 0
        for jj in range(ticks_per_test):
            state_old = agent.get_state(market)
            
            #perform random actions based on agent.epsilon, or choose the action
            if randint(0, 200) < epsilon and False:
                action = to_categorical(randint(0, 1), num_classes=2)
            else:
                # predict action based on the old state
                prediction = agent.predict(state_old)
                action = to_categorical(np.argmax(prediction[0]), num_classes=2)
            
#             print("wow")
#             print(state_old)
#             print(action)
            
            
            reward = 0
            if np.argmax(action) == 0:
                if not market.position:
                    market.buy()
                    #print("Buy ", market.get_price(market.curr_index))
                #else:
                    #print(market.get_price(market.curr_index))
            if np.argmax(action) == 1:
                if market.position:
                    reward = market.sell()
                    #print("Sell ", market.get_price(market.curr_index))
                #else:
                    #print(market.get_price(market.curr_index))
                    
            
            market.tick()
            state_new = agent.get_state(market)
            
            total_reward += reward
            agent.remember(state_old, action, reward, state_new)
            agent.train_single(state_old, action, reward, state_new)
            
        agent.train()
        print('Game', ii, '      Score:', total_reward, '      Epsilon:', epsilon)
        total_reward_old = total_reward
        
        batch = []
        #step = 0.02
        steps = 150
        xmin = -1
        xmax = 2
        ymin = -1
        ymax = 2
        vec = np.linspace(xmin, xmax, steps)
        #for x in np.arange(0.0, 1.0, step):
        for x in vec:
            #for y in np.arange(0.0, 1.0, step):
            for y in vec:
                batch.append(np.asarray([y, x]))
        q = agent.model.predict_on_batch(np.asarray(batch))
        X,Y = np.meshgrid(vec,vec)
        
        z1 = []
        z2 = []
        for ii in range(steps):
            Z1 = []
            Z2 = []
            for p in q[steps*(ii):steps*(ii+1)]:
                Z1.append(p[0])
                Z2.append(p[1])
            z1.append(Z1)
            z2.append(Z2)
        z1 = np.reshape(z1,(1,steps**2))[0]
        z2 = np.reshape(z2,(1,steps**2))[0]
#         z1 = np.sign(z1-0.5)
#         z2 = np.sign(z2-0.5)
            
        z = z1-z2
        chart.chart1.data[0].x = np.reshape(X,(1,steps**2))[0]
        chart.chart1.data[0].y = np.reshape(Y,(1,steps**2))[0]
        chart.chart1.data[0].z = z
#         chart.chart2.data[0].x = np.reshape(X,(1,steps**2))[0]
#         chart.chart2.data[0].y = np.reshape(Y,(1,steps**2))[0]
#         chart.chart2.data[0].z = z2

        
run(chart)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Game 0       Score: 11.0       Epsilon: 0
Game 1       Score: 11.0       Epsilon: 0
Game 2       Score: 4.0       Epsilon: 0
Game 3       Score: 1.0       Epsilon: 0
Game 4       Score: 2.0       Epsilon: 0
Game 5       Score: 1.0       Epsilon: 0
Game 6       Score: 2.0       Epsilon: 0
Game 7       Score: 3.0       Epsilon: 0
Game 8       Score: 2.0       Epsilon: 0
Game 9       Score: 3.0       Epsilon: 0
Game 10       Score: 5.0       Epsilon: 0
Game 11       Score: 6.0       Epsilon: 0
Game 12       Score: 6.0       Epsilon: 0
Game 13       Score: 7.0       Epsilon: 0
Game 14       Score: 6.0       Epsilon: 0
Game 15       Score: 8.0       Epsilon: 0
Game 16       Score: 7.0       Epsilon: 0
Game 17       Score: 7.0       Epsilon: 0
Game 18       Score: 6.0       Epsilon: 0
Game 19       Score: 7.0       Epsilon: 0
Game 20       Score: 5.0       Epsilon: 0
Game 2

Game 176       Score: 10.0       Epsilon: 0
Game 177       Score: 10.0       Epsilon: 0
Game 178       Score: 11.0       Epsilon: 0
Game 179       Score: 10.0       Epsilon: 0
Game 180       Score: 10.0       Epsilon: 0
Game 181       Score: 11.0       Epsilon: 0
Game 182       Score: 10.0       Epsilon: 0
Game 183       Score: 10.0       Epsilon: 0
Game 184       Score: 11.0       Epsilon: 0
Game 185       Score: 10.0       Epsilon: 0
Game 186       Score: 10.0       Epsilon: 0
Game 187       Score: 10.0       Epsilon: 0
Game 188       Score: 11.0       Epsilon: 0
Game 189       Score: 10.0       Epsilon: 0
Game 190       Score: 11.0       Epsilon: 0
Game 191       Score: 10.0       Epsilon: 0
Game 192       Score: 11.0       Epsilon: 0
Game 193       Score: 11.0       Epsilon: 0
Game 194       Score: 10.0       Epsilon: 0
Game 195       Score: 10.0       Epsilon: 0
Game 196       Score: 11.0       Epsilon: 0
Game 197       Score: 10.0       Epsilon: 0
Game 198       Score: 11.0      

Game 363       Score: 11.0       Epsilon: 0
Game 364       Score: 10.0       Epsilon: 0
Game 365       Score: 10.0       Epsilon: 0
Game 366       Score: 10.0       Epsilon: 0
Game 367       Score: 11.0       Epsilon: 0
Game 368       Score: 10.0       Epsilon: 0
Game 369       Score: 10.0       Epsilon: 0
Game 370       Score: 11.0       Epsilon: 0
Game 371       Score: 10.0       Epsilon: 0
Game 372       Score: 11.0       Epsilon: 0
Game 373       Score: 10.0       Epsilon: 0
Game 374       Score: 10.0       Epsilon: 0
Game 375       Score: 11.0       Epsilon: 0
Game 376       Score: 11.0       Epsilon: 0
Game 377       Score: 10.0       Epsilon: 0
Game 378       Score: 11.0       Epsilon: 0
Game 379       Score: 11.0       Epsilon: 0
Game 380       Score: 10.0       Epsilon: 0
Game 381       Score: 11.0       Epsilon: 0
Game 382       Score: 10.0       Epsilon: 0
Game 383       Score: 10.0       Epsilon: 0
Game 384       Score: 11.0       Epsilon: 0
Game 385       Score: 10.0      